<p>
 <img src="ayiti_analytics.png" alt="Ayiti Analytics"width=400 />
</p>

##                                       Ayiti Analytics Final Project


## **A New Social Life** 

* **By: Shelsy DALCIDE**
                                                                                                        July 22, 2020


## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## **Project Description:** <a name="introduction"></a>

It is not easy for someone to leave his country of origin to go live elsewhere a young man would like in Chicago IL to rebuild his life 
This project will be useful for people wishing to leave their countries to live in Chicago IL.
He would like to have additional information so quickly regarding districts such as: 
* **The average salary of a person living in Chicago IL**
* **The price of houses their variations**
* **The community zone**
* **The level of security.**

This problem is interesting for him because he will make new experiences, he will have to adapt to this new home which will allow him to learn more about himself. After our Analysis in fact, he will admit his ability to undertake anything, anywhere.
And he will feel at home in an entirely different place.
Becoming aware of his possibilities will make him more determined to succeed.
And discovering different places will help him to think more creatively.



## Data Prepocessing <a name="data"></a>

In this cell we take our various data either by internet or by Excel document.

# Import our library

In [210]:
#import our library
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import time

import json

import requests
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
from selenium import webdriver

import matplotlib.cm as cm
import matplotlib.colors as colors
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import re
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from math import pi

from sklearn.cluster import KMeans

from sklearn.datasets.samples_generator import make_blobs
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim 
import folium

In [211]:
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)

# Our First Web Scrapping for the Population of Illinois

browser = webdriver.Chrome()

#Breaking down the url in 2 part to be able to insert the page number
url_part_1 = 'http://www.city-data.com/city/Illinois.html'

browser.get(url_part_1)
soup = BeautifulSoup(browser.page_source,'html.parser')
result = soup.find_all(id= 'cityTAB')

len(result)

# with a loop we will have our DataFrame
rows = result[1].find_all('tr')
city_name =[]
population =[]
for row in rows:
    tds =row.find_all('td')
    if len(tds) ==3:        
        city_name.append(tds[1].get_text().strip())
        population.append(tds[2].get_text().strip())
df_population =pd.DataFrame(dict(city_name=city_name,population=population))

    

#let's see the dataset of our population with 
df_population

df_population.isnull().sum()

#here we save the dataset of our city with population
df_population.to_csv('Data/populationIL.csv',index=False)

## Methodology <a name="methodology"></a>


The City of Chicago, is the most populous city in the U.S. state of Illinois, and the third-most-populous city in the United States. With an estimated population of 2,693,976 in 2019, it is also the most populous city in the Midwestern United States.

Our job is to help someone moving to Chicago to find a house in a safe area so that they are comfortable and at an adequate price after getting information about the wages of people living in Chicago. 

* For our work we did the web scrapping to extract data needed for our Analysis. 
* Recommend him what salary he could get living in Chicago depending on the town.
* We would like to have some data found in foursquare so we can see some closer to home such as (church, school, restaurant,     movie theater, super market...)
* We would like to establish a linear regression on the prices of the houses. 
* Make a prediction to see how they'll change in a few years.
* Show him the venues (church, school, restaurant, cinema, supermarket...) near his home via a map. 

Since our dataset contains the house prices per month from 1996 to 2020, we will filter the columns of the last two years from 2018 to 2020.

* Remove missing values.
* Count how many homes sold in each area.
* Count how many homes per month and year.



## Dataset of Population of Illinois saved and load

In [212]:
#load the dataset of population of illinois
df_population_official = pd.read_csv('Data/populationIL.csv')

In [213]:
#see the dataset of population of illinois
df_population_official.head()

,city_name,population
0,"Abingdon, IL","3,226"
1,"Adair, IL",210
2,Addieville,243
3,"Addison, IL","37,297"
4,"Albany, IL",883


In [214]:
# here we replace Illinois in iL
df_population_official.city_name=df_population_official.city_name.str.replace("Illinois","IL")
df_population_official.head()

,city_name,population
0,"Abingdon, IL","3,226"
1,"Adair, IL",210
2,Addieville,243
3,"Addison, IL","37,297"
4,"Albany, IL",883


In [215]:
# we can see the first row
df_population_official.iloc[0,:].city_name

'Abingdon, IL'

In [216]:
df_population_official.city_name.str.strip()

0                   Abingdon, IL
1                      Adair, IL
2                     Addieville
3                    Addison, IL
4                     Albany, IL
5                         Albers
6                     Albion, IL
7                      Aledo, IL
8                         Alexis
9                  Algonquin, IL
10                  Alhambra, IL
11                 Allendale, IL
12                  Allerton, IL
13                      Alma, IL
14                       Alorton
15                     Alpha, IL
16                         Alsey
17                         Alsip
18                  Altamont, IL
19                     Alto Pass
20                    Altona, IL
21                     Alton, IL
22                         Alvan
23                     Alvin, IL
24                     Amboy, IL
25                 Andalusia, IL
26                   Andover, IL
27                       Annawan
28                      Anna, IL
29                   Antioch, IL
30        

In [217]:
df_population_official["city_name"]=df_population_official["city_name"].str.strip()

In [218]:
df_population_official.head()

,city_name,population
0,"Abingdon, IL","3,226"
1,"Adair, IL",210
2,Addieville,243
3,"Addison, IL","37,297"
4,"Albany, IL",883


In [219]:
df_population_official.describe()

,city_name,population
count,1203,1203
unique,1203,1024
top,"Virginia, IL",295
freq,1,6


# Our Second Web Scrapping for the salary of people by cities

#Breaking down the url of our seconde site in 2 part to be able to insert the page number
url_part_2 = 'https://en.wikipedia.org/wiki/List_of_Illinois_locations_by_per_capita_income'

browser.get(url_part_2)
soup = BeautifulSoup(browser.page_source,'html.parser')
result2 =soup.find_all('ol')

lis = result2[0].find_all('li')

city_name,city_income = [],[]
for li in lis:
    print(li.get_text().strip())
    city_name.append(li.get_text().strip().split('–')[0])
    city_income.append(li.get_text().strip().split('–')[-1])
    
df_income = pd.DataFrame(dict(city_name=city_name,city_income=city_income))

#see the dataset
df_income

#we save the dataset who's about the wages of people in this city
df_income.to_csv('Data/City_income.csv',index=False)

# Dataset of Salary  of Illinois saved and load

In [220]:
df_income_official= pd.read_csv('Data/City_income.csv')

In [221]:
df_income_official.head()

,city_name,city_income
0,Winnetka,"$98,139"
1,Kenilworth,"$97,381"
2,Glencoe,"$94,900"
3,Inverness,"$91,623"
4,Barrington Hills,"$91,555"


In [222]:
df_income_official.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1314 entries, 0 to 1313
Data columns (total 2 columns):
city_name      1314 non-null object
city_income    1314 non-null object
dtypes: object(2)
memory usage: 20.7+ KB


# Convert the column city_income into float
* For make Analysis
* And make some calcul (Chart)

In [223]:

def convert_to_float(x):
    x= x.replace(',','')
    x=re.findall('[0-9]+', x)
    return float(x[0])
    
    
    

In [224]:
df_income_official.city_income=df_income_official.city_income.apply(lambda x: convert_to_float(x))

In [225]:
df_income_official.tail()

,city_name,city_income
1309,"Brooklyn, Illinois",7944.0
1310,"Fidelity, Illinois",7798.0
1311,"Valley City, Illinois",6833.0
1312,"Kaskaskia, Illinois",0.0
1313,"Ohlman, Illinois",0.0


In [226]:
df_income_official2 = df_income_official[df_income_official.city_income >0]

In [227]:
df_income_official2.isnull().sum()

city_name      0
city_income    0
dtype: int64

In [228]:
df_income_official2.head()

,city_name,city_income
0,Winnetka,98139.0
1,Kenilworth,97381.0
2,Glencoe,94900.0
3,Inverness,91623.0
4,Barrington Hills,91555.0


# Here we just replace all rows who has Illinois in IL in the dataset of the salary (income)
* For merge our dataset income and the dataaset population
* For make our Analysis

In [229]:
#Here we just replace all rows who has Illinois in IL
df_income_official2.city_name=df_income_official2.city_name.str.replace("Illinois","IL") 

In [230]:
df_income_official2.sort_values("city_name",ascending=False)

,city_name,city_income
673,"Zion, IL",17730.0
1129,"Zeigler, IL",13781.0
178,"Yorkville, IL",24514.0
635,"Yates City, IL",18036.0
611,"Yale, IL",18199.0
773,"Xenia, IL",16944.0
830,"Wyoming, IL",16574.0
782,"Wyanet, IL",16888.0
458,"Worth, IL",19449.0
582,"Worden, IL",18485.0


In [231]:
df_income_official2.iloc[930,:].city_name = 'Abingdon, IL'

In [232]:
df_income_official2.city_name.str.strip()

0                          Winnetka
1                        Kenilworth
2                           Glencoe
3                         Inverness
4                  Barrington Hills
5                       Lake Forest
6                         Oak Brook
7                  North Barrington
8                  South Barrington
9                     Highland Park
10                       Riverwoods
11                         Wilmette
12                     Lincolnshire
13                       Northfield
14                         Hinsdale
15                  Lake Barrington
16                       Long Grove
17                        Deer Park
18                       Burr Ridge
19                     Trout Valley
20                      Medinah, IL
21                        Wayne, IL
22                   Lake Bluff, IL
23                  Bull Valley, IL
24                             Golf
25                         Glenview
26                  Tower Lakes, IL
27                      Kild

In [233]:
df_income_official2["city_name"]=df_income_official2["city_name"].str.strip()

C:\Users\bootcamp\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# Let's merge two dataset 
* The dataset of population 
* The dataset of the income of each city

In [234]:
df_population_official.head()

,city_name,population
0,"Abingdon, IL","3,226"
1,"Adair, IL",210
2,Addieville,243
3,"Addison, IL","37,297"
4,"Albany, IL",883


In [235]:
df_income_official2.head()

,city_name,city_income
0,Winnetka,98139.0
1,Kenilworth,97381.0
2,Glencoe,94900.0
3,Inverness,91623.0
4,Barrington Hills,91555.0


In [236]:
# merge The dataset of population The dataset of the income of each city
df_income_population_merged = pd.merge(df_population_official, df_income_official2, how="inner", on="city_name")
df_income_population_merged.head()

,city_name,population,city_income
0,"Abingdon, IL","3,226",15711.0
1,"Addison, IL","37,297",21201.0
2,"Albany, IL",883,18780.0
3,"Albion, IL","1,957",14747.0
4,"Aledo, IL","3,588",18498.0


In [237]:
df_income_population_merged.dtypes

city_name       object
population      object
city_income    float64
dtype: object

In [238]:
df_income_population_merged.population=df_income_population_merged.population.apply(lambda x: convert_to_float(x))

In [239]:
df_income_population_merged.dtypes

city_name       object
population     float64
city_income    float64
dtype: object

In [240]:
df_income_population_merged.isnull().sum()

city_name      0
population     0
city_income    0
dtype: int64

## Load our Dataset of The price of Houses of all state in USA

In [241]:
#load our dataset
df_city = pd.read_csv('Data/City_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_mon.csv')

In [242]:
# let's see the dataset of the city
df_city.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,CountyName,1/31/1996,2/29/1996,3/31/1996,4/30/1996,5/31/1996,6/30/1996,7/31/1996,8/31/1996,9/30/1996,10/31/1996,11/30/1996,12/31/1996,1/31/1997,2/28/1997,3/31/1997,4/30/1997,5/31/1997,6/30/1997,7/31/1997,8/31/1997,9/30/1997,10/31/1997,11/30/1997,12/31/1997,1/31/1998,2/28/1998,3/31/1998,4/30/1998,5/31/1998,6/30/1998,7/31/1998,8/31/1998,9/30/1998,10/31/1998,11/30/1998,12/31/1998,1/31/1999,2/28/1999,3/31/1999,4/30/1999,5/31/1999,6/30/1999,7/31/1999,8/31/1999,9/30/1999,10/31/1999,11/30/1999,12/31/1999,1/31/2000,2/29/2000,3/31/2000,4/30/2000,5/31/2000,6/30/2000,7/31/2000,8/31/2000,9/30/2000,10/31/2000,11/30/2000,12/31/2000,1/31/2001,2/28/2001,3/31/2001,4/30/2001,5/31/2001,6/30/2001,7/31/2001,8/31/2001,9/30/2001,10/31/2001,11/30/2001,12/31/2001,1/31/2002,2/28/2002,3/31/2002,4/30/2002,5/31/2002,6/30/2002,7/31/2002,8/31/2002,9/30/2002,10/31/2002,11/30/2002,12/31/2002,1/31/2003,2/28/2003,3/31/2003,4/30/2003,5/31/2003,6/30/2003,7/31/2003,8/31/2003,9/30/2003,10/31/2003,11/30/2003,12/31/2003,1/31/2004,2/29/2004,3/31/2004,4/30/2004,5/31/2004,6/30/2004,7/31/2004,8/31/2004,9/30/2004,10/31/2004,11/30/2004,12/31/2004,1/31/2005,2/28/2005,3/31/2005,4/30/2005,5/31/2005,6/30/2005,7/31/2005,8/31/2005,9/30/2005,10/31/2005,11/30/2005,12/31/2005,1/31/2006,2/28/2006,3/31/2006,4/30/2006,5/31/2006,6/30/2006,7/31/2006,8/31/2006,9/30/2006,10/31/2006,11/30/2006,12/31/2006,1/31/2007,2/28/2007,3/31/2007,4/30/2007,5/31/2007,6/30/2007,7/31/2007,8/31/2007,9/30/2007,10/31/2007,11/30/2007,12/31/2007,1/31/2008,2/29/2008,3/31/2008,4/30/2008,5/31/2008,6/30/2008,7/31/2008,8/31/2008,9/30/2008,10/31/2008,11/30/2008,12/31/2008,1/31/2009,2/28/2009,3/31/2009,4/30/2009,5/31/2009,6/30/2009,7/31/2009,8/31/2009,9/30/2009,10/31/2009,11/30/2009,12/31/2009,1/31/2010,2/28/2010,3/31/2010,4/30/2010,5/31/2010,6/30/2010,7/31/2010,8/31/2010,9/30/2010,10/31/2010,11/30/2010,12/31/2010,1/31/2011,2/28/2011,3/31/2011,4/30/2011,5/31/2011,6/30/2011,7/31/2011,8/31/2011,9/30/2011,10/31/2011,11/30/2011,12/31/2011,1/31/2012,2/29/2012,3/31/2012,4/30/2012,5/31/2012,6/30/2012,7/31/2012,8/31/2012,9/30/2012,10/31/2012,11/30/2012,12/31/2012,1/31/2013,2/28/2013,3/31/2013,4/30/2013,5/31/2013,6/30/2013,7/31/2013,8/31/2013,9/30/2013,10/31/2013,11/30/2013,12/31/2013,1/31/2014,2/28/2014,3/31/2014,4/30/2014,5/31/2014,6/30/2014,7/31/2014,8/31/2014,9/30/2014,10/31/2014,11/30/2014,12/31/2014,1/31/2015,2/28/2015,3/31/2015,4/30/2015,5/31/2015,6/30/2015,7/31/2015,8/31/2015,9/30/2015,10/31/2015,11/30/2015,12/31/2015,1/31/2016,2/29/2016,3/31/2016,4/30/2016,5/31/2016,6/30/2016,7/31/2016,8/31/2016,9/30/2016,10/31/2016,11/30/2016,12/31/2016,1/31/2017,2/28/2017,3/31/2017,4/30/2017,5/31/2017,6/30/2017,7/31/2017,8/31/2017,9/30/2017,10/31/2017,11/30/2017,12/31/2017,1/31/2018,2/28/2018,3/31/2018,4/30/2018,5/31/2018,6/30/2018,7/31/2018,8/31/2018,9/30/2018,10/31/2018,11/30/2018,12/31/2018,1/31/2019,2/28/2019,3/31/2019,4/30/2019,5/31/2019,6/30/2019,7/31/2019,8/31/2019,9/30/2019,10/31/2019,11/30/2019,12/31/2019,1/31/2020,2/29/2020,3/31/2020,4/30/2020,5/31/2020,6/30/2020
0,6181,0,New York,City,NY,NY,New York-Newark-Jersey City,Queens County,360437.0,359463.0,358991.0,358191.0,358006.0,357854.0,357716.0,357541.0,357568.0,357854.0,358257.0,358886.0,359974.0,362104.0,364267.0,365939.0,366885.0,368112.0,369662.0,371419.0,373144.0,374783.0,376990.0,379345.0,381657.0,382874.0,383875.0,384978.0,386668.0,388448.0,390140.0,392479.0,395063.0,398202.0,400823.0,403402.0,405454.0,407952.0,410651.0,414196.0,417955.0,422086.0,426942.0,431339.0,435845.0,439448.0,442929.0,446276.0,449993.0,455462.0,461741.0,468116.0,473757.0,479034.0,484207.0,488804.0,493498.0,498601.0,504559.0,510106.0,515497.0,521559.0,529366.0,536985.0,543096.0,547042.0,551095.0,555429.0,559727.0,564229.0,569318.0,575446.0,582020.0,583059.0,579219.0,574232.0,573701.0,576189.0,579606.0,584746.0,591156.0,596701.0,600351.0,603863.0,607389.0,614338.0,623682.0,634586.0,643339.0,654226.0,661277.0,666804.0,667521.0,670286

In [243]:
df_city.columns.to_list()

['RegionID',
 'SizeRank',
 'RegionName',
 'RegionType',
 'StateName',
 'State',
 'Metro',
 'CountyName',
 '1/31/1996',
 '2/29/1996',
 '3/31/1996',
 '4/30/1996',
 '5/31/1996',
 '6/30/1996',
 '7/31/1996',
 '8/31/1996',
 '9/30/1996',
 '10/31/1996',
 '11/30/1996',
 '12/31/1996',
 '1/31/1997',
 '2/28/1997',
 '3/31/1997',
 '4/30/1997',
 '5/31/1997',
 '6/30/1997',
 '7/31/1997',
 '8/31/1997',
 '9/30/1997',
 '10/31/1997',
 '11/30/1997',
 '12/31/1997',
 '1/31/1998',
 '2/28/1998',
 '3/31/1998',
 '4/30/1998',
 '5/31/1998',
 '6/30/1998',
 '7/31/1998',
 '8/31/1998',
 '9/30/1998',
 '10/31/1998',
 '11/30/1998',
 '12/31/1998',
 '1/31/1999',
 '2/28/1999',
 '3/31/1999',
 '4/30/1999',
 '5/31/1999',
 '6/30/1999',
 '7/31/1999',
 '8/31/1999',
 '9/30/1999',
 '10/31/1999',
 '11/30/1999',
 '12/31/1999',
 '1/31/2000',
 '2/29/2000',
 '3/31/2000',
 '4/30/2000',
 '5/31/2000',
 '6/30/2000',
 '7/31/2000',
 '8/31/2000',
 '9/30/2000',
 '10/31/2000',
 '11/30/2000',
 '12/31/2000',
 '1/31/2001',
 '2/28/2001',
 '3/31/2001'

# Filtre all IL (Illinois) in Data of houses Price

In [244]:
#trill the lines of city in the from column 'RegionType' and IL in the from column 'StateName'.
df_city_filtred = df_city[(df_city["RegionType"] == 'City')& (df_city["StateName"] == 'IL') ]
df_city_filtred.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,CountyName,1/31/1996,2/29/1996,3/31/1996,4/30/1996,5/31/1996,6/30/1996,7/31/1996,8/31/1996,9/30/1996,10/31/1996,11/30/1996,12/31/1996,1/31/1997,2/28/1997,3/31/1997,4/30/1997,5/31/1997,6/30/1997,7/31/1997,8/31/1997,9/30/1997,10/31/1997,11/30/1997,12/31/1997,1/31/1998,2/28/1998,3/31/1998,4/30/1998,5/31/1998,6/30/1998,7/31/1998,8/31/1998,9/30/1998,10/31/1998,11/30/1998,12/31/1998,1/31/1999,2/28/1999,3/31/1999,4/30/1999,5/31/1999,6/30/1999,7/31/1999,8/31/1999,9/30/1999,10/31/1999,11/30/1999,12/31/1999,1/31/2000,2/29/2000,3/31/2000,4/30/2000,5/31/2000,6/30/2000,7/31/2000,8/31/2000,9/30/2000,10/31/2000,11/30/2000,12/31/2000,1/31/2001,2/28/2001,3/31/2001,4/30/2001,5/31/2001,6/30/2001,7/31/2001,8/31/2001,9/30/2001,10/31/2001,11/30/2001,12/31/2001,1/31/2002,2/28/2002,3/31/2002,4/30/2002,5/31/2002,6/30/2002,7/31/2002,8/31/2002,9/30/2002,10/31/2002,11/30/2002,12/31/2002,1/31/2003,2/28/2003,3/31/2003,4/30/2003,5/31/2003,6/30/2003,7/31/2003,8/31/2003,9/30/2003,10/31/2003,11/30/2003,12/31/2003,1/31/2004,2/29/2004,3/31/2004,4/30/2004,5/31/2004,6/30/2004,7/31/2004,8/31/2004,9/30/2004,10/31/2004,11/30/2004,12/31/2004,1/31/2005,2/28/2005,3/31/2005,4/30/2005,5/31/2005,6/30/2005,7/31/2005,8/31/2005,9/30/2005,10/31/2005,11/30/2005,12/31/2005,1/31/2006,2/28/2006,3/31/2006,4/30/2006,5/31/2006,6/30/2006,7/31/2006,8/31/2006,9/30/2006,10/31/2006,11/30/2006,12/31/2006,1/31/2007,2/28/2007,3/31/2007,4/30/2007,5/31/2007,6/30/2007,7/31/2007,8/31/2007,9/30/2007,10/31/2007,11/30/2007,12/31/2007,1/31/2008,2/29/2008,3/31/2008,4/30/2008,5/31/2008,6/30/2008,7/31/2008,8/31/2008,9/30/2008,10/31/2008,11/30/2008,12/31/2008,1/31/2009,2/28/2009,3/31/2009,4/30/2009,5/31/2009,6/30/2009,7/31/2009,8/31/2009,9/30/2009,10/31/2009,11/30/2009,12/31/2009,1/31/2010,2/28/2010,3/31/2010,4/30/2010,5/31/2010,6/30/2010,7/31/2010,8/31/2010,9/30/2010,10/31/2010,11/30/2010,12/31/2010,1/31/2011,2/28/2011,3/31/2011,4/30/2011,5/31/2011,6/30/2011,7/31/2011,8/31/2011,9/30/2011,10/31/2011,11/30/2011,12/31/2011,1/31/2012,2/29/2012,3/31/2012,4/30/2012,5/31/2012,6/30/2012,7/31/2012,8/31/2012,9/30/2012,10/31/2012,11/30/2012,12/31/2012,1/31/2013,2/28/2013,3/31/2013,4/30/2013,5/31/2013,6/30/2013,7/31/2013,8/31/2013,9/30/2013,10/31/2013,11/30/2013,12/31/2013,1/31/2014,2/28/2014,3/31/2014,4/30/2014,5/31/2014,6/30/2014,7/31/2014,8/31/2014,9/30/2014,10/31/2014,11/30/2014,12/31/2014,1/31/2015,2/28/2015,3/31/2015,4/30/2015,5/31/2015,6/30/2015,7/31/2015,8/31/2015,9/30/2015,10/31/2015,11/30/2015,12/31/2015,1/31/2016,2/29/2016,3/31/2016,4/30/2016,5/31/2016,6/30/2016,7/31/2016,8/31/2016,9/30/2016,10/31/2016,11/30/2016,12/31/2016,1/31/2017,2/28/2017,3/31/2017,4/30/2017,5/31/2017,6/30/2017,7/31/2017,8/31/2017,9/30/2017,10/31/2017,11/30/2017,12/31/2017,1/31/2018,2/28/2018,3/31/2018,4/30/2018,5/31/2018,6/30/2018,7/31/2018,8/31/2018,9/30/2018,10/31/2018,11/30/2018,12/31/2018,1/31/2019,2/28/2019,3/31/2019,4/30/2019,5/31/2019,6/30/2019,7/31/2019,8/31/2019,9/30/2019,10/31/2019,11/30/2019,12/31/2019,1/31/2020,2/29/2020,3/31/2020,4/30/2020,5/31/2020,6/30/2020
3,17426,3,Chicago,City,IL,IL,Chicago-Naperville-Elgin,Cook County,253876.0,253589.0,253081.0,253006.0,252374.0,252573.0,252332.0,253093.0,254162.0,255668.0,257337.0,258530.0,259775.0,260885.0,262182.0,262431.0,263808.0,264037.0,264653.0,262965.0,260268.0,256491.0,253036.0,251213.0,250259.0,249898.0,250179.0,250712.0,251187.0,251472.0,252283.0,254982.0,259410.0,265232.0,270445.0,274388.0,276702.0,278707.0,280295.0,282288.0,283784.0,286614.0,289738.0,292397.0,293870.0,294908.0,296727.0,298907.0,302335.0,305843.0,309578.0,312927.0,316501.0,319754.0,322912.0,326044.0,329285.0,332594.0,336227.0,339967.0,343205.0,345860.0,348677.0,351725.0,355106.0,358278.0,360932.0,363662.0,366343.0,369331.0,372011.0,374215.0,376318.0,378656.0,381519.0,383671.0,385885.0,387569.0,390458.0,393399.0,396848.0,399540.0,401619.0,403969.0,406545.0,409717.0,412291.0,415164.0,416976.0,419098.0,420580.0,422580.0,424561.0,427137.0,42

In [245]:
df_city_filtred_new =df_city_filtred[['SizeRank','RegionName','RegionType','StateName','State','Metro','CountyName','1/31/2018',
 '2/28/2018',
 '3/31/2018',
 '4/30/2018',
 '5/31/2018',
 '6/30/2018',
 '7/31/2018',
 '8/31/2018',
 '9/30/2018',
 '10/31/2018',
 '11/30/2018',
 '12/31/2018',
 '1/31/2019',
 '2/28/2019',
 '3/31/2019',
 '4/30/2019',
 '5/31/2019',
 '6/30/2019',
 '7/31/2019',
 '8/31/2019',
 '9/30/2019',
 '10/31/2019',
 '11/30/2019',
 '12/31/2019',
 '1/31/2020',
 '2/29/2020',
 '3/31/2020',
 '4/30/2020',
 '5/31/2020',
 '6/30/2020']]

In [246]:
df_city_filtred_new.head()

,SizeRank,RegionName,RegionType,StateName,State,Metro,CountyName,1/31/2018,2/28/2018,3/31/2018,4/30/2018,5/31/2018,6/30/2018,7/31/2018,8/31/2018,9/30/2018,10/31/2018,11/30/2018,12/31/2018,1/31/2019,2/28/2019,3/31/2019,4/30/2019,5/31/2019,6/30/2019,7/31/2019,8/31/2019,9/30/2019,10/31/2019,11/30/2019,12/31/2019,1/31/2020,2/29/2020,3/31/2020,4/30/2020,5/31/2020,6/30/2020
3,3,Chicago,City,IL,IL,Chicago-Naperville-Elgin,Cook County,463386.0,465437.0,464958.0,464273.0,464259.0,465020.0,464935.0,464867.0,465305.0,465988.0,466946.0,467162.0,465606.0,464067.0,463288.0,463675.0,464009.0,463854.0,464049.0,463761.0,463247.0,462900.0,462366.0,461987.0,462121.0,462767.0,463652.0,463953.0,464035.0,464325.0
143,144,Aurora,City,IL,IL,Chicago-Naperville-Elgin,Kane County,274618.0,274759.0,275212.0,275987.0,277095.0,278440.0,279595.0,281001.0,282239.0,283256.0,283179.0,283146.0,283121.0,284171.0,284427.0,284632.0,284828.0,284827.0,284687.0,283916.0,283674.0,283272.0,283405.0,283468.0,283776.0,283589.0,283913.0,284148.0,284302.0,284554.0
156,157,Rockford,City,IL,IL,Rockford,Winnebago County,146230.0,147804.0,147581.0,147242.0,146828.0,147640.0,148972.0,149638.0,150206.0,150497.0,150590.0,150926.0,151328.0,151658.0,153037.0,154262.0,155541.0,155693.0,155650.0,155876.0,155825.0,156069.0,156407.0,156595.0,156780.0,156682.0,156543.0,156585.0,156714.0,157083.0
190,192,Naperville,City,IL,IL,Chicago-Naperville-Elgin,DuPage County,533985.0,532568.0,532309.0,532955.0,534572.0,536828.0,538351.0,540067.0,541389.0,541753.0,541398.0,539985.0,538693.0,538827.0,538426.0,539149.0,539761.0,540105.0,539815.0,538924.0,538675.0,538591.0,538599.0,539199.0,540000.0,540545.0,541031.0,540447.0,539459.0,538502.0
279,284,Springfield,City,IL,IL,Springfield,Sangamon County,215954.0,216082.0,216545.0,217093.0,217582.0,217689.0,217527.0,217497.0,217351.0,217410.0,217560.0,217926.0,218055.0,218129.0,218126.0,218329.0,218543.0,218503.0,218546.0,218485.0,218774.0,218988.0,219187.0,219269.0,219530.0,219857.0,220172.0,220354.0,220617.0,221303.0


In [247]:
df_city_filtred_new.rename(columns = {'CountyName':'county_name'}, inplace = True)

In [248]:
df_city_filtred_new.head()

,SizeRank,RegionName,RegionType,StateName,State,Metro,county_name,1/31/2018,2/28/2018,3/31/2018,4/30/2018,5/31/2018,6/30/2018,7/31/2018,8/31/2018,9/30/2018,10/31/2018,11/30/2018,12/31/2018,1/31/2019,2/28/2019,3/31/2019,4/30/2019,5/31/2019,6/30/2019,7/31/2019,8/31/2019,9/30/2019,10/31/2019,11/30/2019,12/31/2019,1/31/2020,2/29/2020,3/31/2020,4/30/2020,5/31/2020,6/30/2020
3,3,Chicago,City,IL,IL,Chicago-Naperville-Elgin,Cook County,463386.0,465437.0,464958.0,464273.0,464259.0,465020.0,464935.0,464867.0,465305.0,465988.0,466946.0,467162.0,465606.0,464067.0,463288.0,463675.0,464009.0,463854.0,464049.0,463761.0,463247.0,462900.0,462366.0,461987.0,462121.0,462767.0,463652.0,463953.0,464035.0,464325.0
143,144,Aurora,City,IL,IL,Chicago-Naperville-Elgin,Kane County,274618.0,274759.0,275212.0,275987.0,277095.0,278440.0,279595.0,281001.0,282239.0,283256.0,283179.0,283146.0,283121.0,284171.0,284427.0,284632.0,284828.0,284827.0,284687.0,283916.0,283674.0,283272.0,283405.0,283468.0,283776.0,283589.0,283913.0,284148.0,284302.0,284554.0
156,157,Rockford,City,IL,IL,Rockford,Winnebago County,146230.0,147804.0,147581.0,147242.0,146828.0,147640.0,148972.0,149638.0,150206.0,150497.0,150590.0,150926.0,151328.0,151658.0,153037.0,154262.0,155541.0,155693.0,155650.0,155876.0,155825.0,156069.0,156407.0,156595.0,156780.0,156682.0,156543.0,156585.0,156714.0,157083.0
190,192,Naperville,City,IL,IL,Chicago-Naperville-Elgin,DuPage County,533985.0,532568.0,532309.0,532955.0,534572.0,536828.0,538351.0,540067.0,541389.0,541753.0,541398.0,539985.0,538693.0,538827.0,538426.0,539149.0,539761.0,540105.0,539815.0,538924.0,538675.0,538591.0,538599.0,539199.0,540000.0,540545.0,541031.0,540447.0,539459.0,538502.0
279,284,Springfield,City,IL,IL,Springfield,Sangamon County,215954.0,216082.0,216545.0,217093.0,217582.0,217689.0,217527.0,217497.0,217351.0,217410.0,217560.0,217926.0,218055.0,218129.0,218126.0,218329.0,218543.0,218503.0,218546.0,218485.0,218774.0,218988.0,219187.0,219269.0,219530.0,219857.0,220172.0,220354.0,220617.0,221303.0


In [249]:
df_city_filtred_new.dtypes

SizeRank         int64
RegionName      object
RegionType      object
StateName       object
State           object
Metro           object
county_name     object
1/31/2018      float64
2/28/2018      float64
3/31/2018      float64
4/30/2018      float64
5/31/2018      float64
6/30/2018      float64
7/31/2018      float64
8/31/2018      float64
9/30/2018      float64
10/31/2018     float64
11/30/2018     float64
12/31/2018     float64
1/31/2019      float64
2/28/2019      float64
3/31/2019      float64
4/30/2019      float64
5/31/2019      float64
6/30/2019      float64
7/31/2019      float64
8/31/2019      float64
9/30/2019      float64
10/31/2019     float64
11/30/2019     float64
12/31/2019     float64
1/31/2020      float64
2/29/2020      float64
3/31/2020      float64
4/30/2020      float64
5/31/2020      float64
6/30/2020      float64
dtype: object

In [270]:
df_city_filtred_new.dropna(inplace = True, axis = 0)

C:\Users\bootcamp\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [271]:
df_city_filtred_new.isnull().sum()

SizeRank       0
RegionName     0
RegionType     0
StateName      0
State          0
Metro          0
county_name    0
1/31/2018      0
2/28/2018      0
3/31/2018      0
4/30/2018      0
5/31/2018      0
6/30/2018      0
7/31/2018      0
8/31/2018      0
9/30/2018      0
10/31/2018     0
11/30/2018     0
12/31/2018     0
1/31/2019      0
2/28/2019      0
3/31/2019      0
4/30/2019      0
5/31/2019      0
6/30/2019      0
7/31/2019      0
8/31/2019      0
9/30/2019      0
10/31/2019     0
11/30/2019     0
12/31/2019     0
1/31/2020      0
2/29/2020      0
3/31/2020      0
4/30/2020      0
5/31/2020      0
6/30/2020      0
dtype: int64

## Load Dataset of Crime 

In [272]:
df_crime = pd.read_csv('Data/2018_ucr_index_arrest-191115T22305341.csv')

In [273]:
# 
df_crime.head()

,year,id,fips_number,county_name,judicial_circuit,region,community_type,percent_rural,population,aggravated_assault,arson,burglary,larceny_theft,motor_vehicle_theft,murder,robbery,sexual_assault,aggravated_assault_rate,arson_rate,burglary_rate,larceny_theft_rate,motor_vehicle_theft_rate,murder_rate,robbery_rate,sexual_assault_rate
0,1982,1,1,Adams,8,Central,Mostly Urban,32.6,71174,45.0,4.0,54.0,326.0,8.0,3.0,6.0,5.0,63.2,5.6,75.9,458.0,11.2,4.2,8.4,7.0
1,1982,2,3,Alexander,1,Southern,Mostly Rural,61.9,12056,75.0,0.0,32.0,57.0,2.0,1.0,10.0,2.0,622.1,0.0,265.4,472.8,16.6,8.3,82.9,16.6
2,1982,3,5,Bond,3,Southern,Mostly Rural,60.7,16132,0.0,0.0,2.0,19.0,3.0,0.0,0.0,0.0,0.0,0.0,12.4,117.8,18.6,0.0,0.0,0.0
3,1982,4,7,Boone,17,Northern - Other,Mostly Urban,19.4,29213,7.0,0.0,41.0,102.0,5.0,0.0,0.0,0.0,24.0,0.0,140.3,349.2,17.1,0.0,0.0,0.0
4,1982,5,9,Brown,8,Central,Mostly Urban,41.1,5588,0.0,0.0,4.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,71.6,161.1,0.0,0.0,0.0,0.0


In [274]:
df_crime.isnull().sum()

year                         0
id                           0
fips_number                  0
county_name                  0
judicial_circuit             0
region                       0
community_type               0
percent_rural                0
population                   0
aggravated_assault          23
arson                       23
burglary                    23
larceny_theft               23
motor_vehicle_theft         23
murder                      23
robbery                     23
sexual_assault              23
aggravated_assault_rate     23
arson_rate                  23
burglary_rate               23
larceny_theft_rate          23
motor_vehicle_theft_rate    23
murder_rate                 23
robbery_rate                23
sexual_assault_rate         23
dtype: int64

In [275]:
df_crime.dropna(inplace=True,axis=0)

In [276]:
df_crime.isnull().sum()

year                        0
id                          0
fips_number                 0
county_name                 0
judicial_circuit            0
region                      0
community_type              0
percent_rural               0
population                  0
aggravated_assault          0
arson                       0
burglary                    0
larceny_theft               0
motor_vehicle_theft         0
murder                      0
robbery                     0
sexual_assault              0
aggravated_assault_rate     0
arson_rate                  0
burglary_rate               0
larceny_theft_rate          0
motor_vehicle_theft_rate    0
murder_rate                 0
robbery_rate                0
sexual_assault_rate         0
dtype: int64

In [277]:
df_crime.head()

,year,id,fips_number,county_name,judicial_circuit,region,community_type,percent_rural,population,aggravated_assault,arson,burglary,larceny_theft,motor_vehicle_theft,murder,robbery,sexual_assault,aggravated_assault_rate,arson_rate,burglary_rate,larceny_theft_rate,motor_vehicle_theft_rate,murder_rate,robbery_rate,sexual_assault_rate
0,1982,1,1,Adams,8,Central,Mostly Urban,32.6,71174,45.0,4.0,54.0,326.0,8.0,3.0,6.0,5.0,63.2,5.6,75.9,458.0,11.2,4.2,8.4,7.0
1,1982,2,3,Alexander,1,Southern,Mostly Rural,61.9,12056,75.0,0.0,32.0,57.0,2.0,1.0,10.0,2.0,622.1,0.0,265.4,472.8,16.6,8.3,82.9,16.6
2,1982,3,5,Bond,3,Southern,Mostly Rural,60.7,16132,0.0,0.0,2.0,19.0,3.0,0.0,0.0,0.0,0.0,0.0,12.4,117.8,18.6,0.0,0.0,0.0
3,1982,4,7,Boone,17,Northern - Other,Mostly Urban,19.4,29213,7.0,0.0,41.0,102.0,5.0,0.0,0.0,0.0,24.0,0.0,140.3,349.2,17.1,0.0,0.0,0.0
4,1982,5,9,Brown,8,Central,Mostly Urban,41.1,5588,0.0,0.0,4.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,71.6,161.1,0.0,0.0,0.0,0.0


In [278]:
#add 'County' in our dataset of the column of county_name
df_crime.county_name = df_crime.county_name + str(' County')

## Let's Merge two dataset:
* The Dataset of Crime in Illinois (IL)
* The Dataset of City

In [279]:
#Let's Merge two dataset Crime and the City
df_city_crime_merge = pd.merge(df_city_filtred_new, df_crime, how="inner", on="county_name")
df_city_crime_merge.tail()


,SizeRank,RegionName,RegionType,StateName,State,Metro,county_name,1/31/2018,2/28/2018,3/31/2018,4/30/2018,5/31/2018,6/30/2018,7/31/2018,8/31/2018,9/30/2018,10/31/2018,11/30/2018,12/31/2018,1/31/2019,2/28/2019,3/31/2019,4/30/2019,5/31/2019,6/30/2019,7/31/2019,8/31/2019,9/30/2019,10/31/2019,11/30/2019,12/31/2019,1/31/2020,2/29/2020,3/31/2020,4/30/2020,5/31/2020,6/30/2020,year,id,fips_number,judicial_circuit,region,community_type,percent_rural,population,aggravated_assault,arson,burglary,larceny_theft,motor_vehicle_theft,murder,robbery,sexual_assault,aggravated_assault_rate,arson_rate,burglary_rate,larceny_theft_rate,motor_vehicle_theft_rate,murder_rate,robbery_rate,sexual_assault_rate
35434,26452,Batchtown,City,IL,IL,St. Louis,Calhoun County,169123.0,168784.0,168637.0,168212.0,167683.0,165661.0,164326.0,163702.0,163936.0,164147.0,162624.0,160916.0,159221.0,158545.0,157507.0,157060.0,157034.0,158704.0,159617.0,160134.0,160100.0,160088.0,160840.0,161634.0,162370.0,162593.0,162932.0,162915.0,162617.0,161525.0,2007,7,13,8,Southern,Completely Rural,100.0,5097,15.0,0.0,10.0,16.0,1.0,0.0,0.0,0.0,294.3,0.0,196.2,313.9,19.6,0.0,0.0,0.0
35435,26452,Batchtown,City,IL,IL,St. Louis,Calhoun County,169123.0,168784.0,168637.0,168212.0,167683.0,165661.0,164326.0,163702.0,163936.0,164147.0,162624.0,160916.0,159221.0,158545.0,157507.0,157060.0,157034.0,158704.0,159617.0,160134.0,160100.0,160088.0,160840.0,161634.0,162370.0,162593.0,162932.0,162915.0,162617.0,161525.0,2008,7,13,8,Southern,Completely Rural,100.0,5024,4.0,0.0,5.0,2.0,1.0,0.0,0.0,0.0,79.6,0.0,99.5,39.8,19.9,0.0,0.0,0.0
35436,26452,Batchtown,City,IL,IL,St. Louis,Calhoun County,169123.0,168784.0,168637.0,168212.0,167683.0,165661.0,164326.0,163702.0,163936.0,164147.0,162624.0,160916.0,159221.0,158545.0,157507.0,157060.0,157034.0,158704.0,159617.0,160134.0,160100.0,160088.0,160840.0,161634.0,162370.0,162593.0,162932.0,162915.0,162617.0,161525.0,2009,7,13,8,Southern,Completely Rural,100.0,5019,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,19.9,39.8,0.0,0.0,0.0
35437,26452,Batchtown,City,IL,IL,St. Louis,Calhoun County,169123.0,168784.0,168637.0,168212.0,167683.0,165661.0,164326.0,163702.0,163936.0,164147.0,162624.0,160916.0,159221.0,158545.0,157507.0,157060.0,157034.0,158704.0,159617.0,160134.0,160100.0,160088.0,160840.0,161634.0,162370.0,162593.0,162932.0,162915.0,162617.0,161525.0,2010,7,13,8,Southern,Completely Rural,100.0,5096,9.0,0.0,3.0,2.0,1.0,0.0,0.0,0.0,176.6,0.0,58.9,39.2,19.6,0.0,0.0,0.0
35438,26452,Batchtown,City,IL,IL,St. Louis,Calhoun County,169123.0,168784.0,168637.0,168212.0,167683.0,165661.0,164326.0,163702.0,163936.0,164147.0,162624.0,160916.0,159221.0,158545.0,157507.0,157060.0,157034.0,158704.0,159617.0,160134.0,160100.0,160088.0,160840.0,161634.0,162370.0,162593.0,162932.0,162915.0,162617.0,161525.0,2011,7,13,8,Southern,Completely Rural,100.0,5048,8.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,158.5,0.0,59.4,39.6,0.0,0.0,0.0,0.0


In [280]:
df_city_crime_merge.rename(columns = {'RegionName':'city_name'}, inplace = True)

In [281]:
# 
df_city_crime_merge = df_city_crime_merge[df_city_crime_merge['year']== 2018]

In [282]:
df_city_crime_merge.head()

,SizeRank,city_name,RegionType,StateName,State,Metro,county_name,1/31/2018,2/28/2018,3/31/2018,4/30/2018,5/31/2018,6/30/2018,7/31/2018,8/31/2018,9/30/2018,10/31/2018,11/30/2018,12/31/2018,1/31/2019,2/28/2019,3/31/2019,4/30/2019,5/31/2019,6/30/2019,7/31/2019,8/31/2019,9/30/2019,10/31/2019,11/30/2019,12/31/2019,1/31/2020,2/29/2020,3/31/2020,4/30/2020,5/31/2020,6/30/2020,year,id,fips_number,judicial_circuit,region,community_type,percent_rural,population,aggravated_assault,arson,burglary,larceny_theft,motor_vehicle_theft,murder,robbery,sexual_assault,aggravated_assault_rate,arson_rate,burglary_rate,larceny_theft_rate,motor_vehicle_theft_rate,murder_rate,robbery_rate,sexual_assault_rate
36,3,Chicago,City,IL,IL,Chicago-Naperville-Elgin,Cook County,463386.0,465437.0,464958.0,464273.0,464259.0,465020.0,464935.0,464867.0,465305.0,465988.0,466946.0,467162.0,465606.0,464067.0,463288.0,463675.0,464009.0,463854.0,464049.0,463761.0,463247.0,462900.0,462366.0,461987.0,462121.0,462767.0,463652.0,463953.0,464035.0,464325.0,2018,16,31,co,Northern - Cook,Completely Urban,0.0,5180493,2455.0,57.0,1266.0,14732.0,2791.0,327.0,1766.0,493.0,47.4,1.1,24.4,284.4,53.9,6.3,34.1,9.5
73,532,Cicero,City,IL,IL,Chicago-Naperville-Elgin,Cook County,200197.0,201097.0,201169.0,202682.0,204383.0,205909.0,207135.0,209094.0,211084.0,212932.0,214843.0,215709.0,216468.0,217232.0,217931.0,218303.0,218121.0,218268.0,218645.0,218506.0,218253.0,217890.0,217622.0,217427.0,217595.0,217717.0,218046.0,217975.0,218267.0,218485.0,2018,16,31,co,Northern - Cook,Completely Urban,0.0,5180493,2455.0,57.0,1266.0,14732.0,2791.0,327.0,1766.0,493.0,47.4,1.1,24.4,284.4,53.9,6.3,34.1,9.5
110,577,Arlington Heights,City,IL,IL,Chicago-Naperville-Elgin,Cook County,480732.0,482346.0,482235.0,481389.0,481597.0,481500.0,480310.0,478409.0,477775.0,477481.0,478557.0,478652.0,477552.0,475912.0,474600.0,474567.0,473892.0,473217.0,472545.0,472095.0,471196.0,470247.0,468855.0,467828.0,466961.0,466909.0,467159.0,467006.0,466924.0,467078.0,2018,16,31,co,Northern - Cook,Completely Urban,0.0,5180493,2455.0,57.0,1266.0,14732.0,2791.0,327.0,1766.0,493.0,47.4,1.1,24.4,284.4,53.9,6.3,34.1,9.5
147,592,Schaumburg,City,IL,IL,Chicago-Naperville-Elgin,Cook County,369183.0,370912.0,370593.0,370393.0,370949.0,372627.0,372686.0,372796.0,372912.0,373366.0,373921.0,374112.0,373279.0,372490.0,372031.0,372216.0,371995.0,371637.0,371771.0,371994.0,371812.0,371584.0,371371.0,371231.0,371167.0,371565.0,372319.0,372580.0,372815.0,372788.0,2018,16,31,co,Northern - Cook,Completely Urban,0.0,5180493,2455.0,57.0,1266.0,14732.0,2791.0,327.0,1766.0,493.0,47.4,1.1,24.4,284.4,53.9,6.3,34.1,9.5
184,629,Evanston,City,IL,IL,Chicago-Naperville-Elgin,Cook County,750793.0,757648.0,760333.0,760313.0,759305.0,758076.0,755856.0,754216.0,754298.0,754766.0,755798.0,756903.0,754806.0,753732.0,752570.0,751984.0,750429.0,749602.0,750106.0,750109.0,748468.0,746183.0,743861.0,741338.0,739523.0,737185.0,735897.0,735250.0,735621.0,735620.0,2018,16,31,co,Northern - Cook,Completely Urban,0.0,5180493,2455.0,57.0,1266.0,14732.0,2791.0,327.0,1766.0,493.0,47.4,1.1,24.4,284.4,53.9,6.3,34.1,9.5


In [283]:
df_city_crime_merge['city_name'] = df_city_crime_merge['city_name'] + ', IL'

In [284]:
df_city_crime_merge.head()

,SizeRank,city_name,RegionType,StateName,State,Metro,county_name,1/31/2018,2/28/2018,3/31/2018,4/30/2018,5/31/2018,6/30/2018,7/31/2018,8/31/2018,9/30/2018,10/31/2018,11/30/2018,12/31/2018,1/31/2019,2/28/2019,3/31/2019,4/30/2019,5/31/2019,6/30/2019,7/31/2019,8/31/2019,9/30/2019,10/31/2019,11/30/2019,12/31/2019,1/31/2020,2/29/2020,3/31/2020,4/30/2020,5/31/2020,6/30/2020,year,id,fips_number,judicial_circuit,region,community_type,percent_rural,population,aggravated_assault,arson,burglary,larceny_theft,motor_vehicle_theft,murder,robbery,sexual_assault,aggravated_assault_rate,arson_rate,burglary_rate,larceny_theft_rate,motor_vehicle_theft_rate,murder_rate,robbery_rate,sexual_assault_rate
36,3,"Chicago, IL",City,IL,IL,Chicago-Naperville-Elgin,Cook County,463386.0,465437.0,464958.0,464273.0,464259.0,465020.0,464935.0,464867.0,465305.0,465988.0,466946.0,467162.0,465606.0,464067.0,463288.0,463675.0,464009.0,463854.0,464049.0,463761.0,463247.0,462900.0,462366.0,461987.0,462121.0,462767.0,463652.0,463953.0,464035.0,464325.0,2018,16,31,co,Northern - Cook,Completely Urban,0.0,5180493,2455.0,57.0,1266.0,14732.0,2791.0,327.0,1766.0,493.0,47.4,1.1,24.4,284.4,53.9,6.3,34.1,9.5
73,532,"Cicero, IL",City,IL,IL,Chicago-Naperville-Elgin,Cook County,200197.0,201097.0,201169.0,202682.0,204383.0,205909.0,207135.0,209094.0,211084.0,212932.0,214843.0,215709.0,216468.0,217232.0,217931.0,218303.0,218121.0,218268.0,218645.0,218506.0,218253.0,217890.0,217622.0,217427.0,217595.0,217717.0,218046.0,217975.0,218267.0,218485.0,2018,16,31,co,Northern - Cook,Completely Urban,0.0,5180493,2455.0,57.0,1266.0,14732.0,2791.0,327.0,1766.0,493.0,47.4,1.1,24.4,284.4,53.9,6.3,34.1,9.5
110,577,"Arlington Heights, IL",City,IL,IL,Chicago-Naperville-Elgin,Cook County,480732.0,482346.0,482235.0,481389.0,481597.0,481500.0,480310.0,478409.0,477775.0,477481.0,478557.0,478652.0,477552.0,475912.0,474600.0,474567.0,473892.0,473217.0,472545.0,472095.0,471196.0,470247.0,468855.0,467828.0,466961.0,466909.0,467159.0,467006.0,466924.0,467078.0,2018,16,31,co,Northern - Cook,Completely Urban,0.0,5180493,2455.0,57.0,1266.0,14732.0,2791.0,327.0,1766.0,493.0,47.4,1.1,24.4,284.4,53.9,6.3,34.1,9.5
147,592,"Schaumburg, IL",City,IL,IL,Chicago-Naperville-Elgin,Cook County,369183.0,370912.0,370593.0,370393.0,370949.0,372627.0,372686.0,372796.0,372912.0,373366.0,373921.0,374112.0,373279.0,372490.0,372031.0,372216.0,371995.0,371637.0,371771.0,371994.0,371812.0,371584.0,371371.0,371231.0,371167.0,371565.0,372319.0,372580.0,372815.0,372788.0,2018,16,31,co,Northern - Cook,Completely Urban,0.0,5180493,2455.0,57.0,1266.0,14732.0,2791.0,327.0,1766.0,493.0,47.4,1.1,24.4,284.4,53.9,6.3,34.1,9.5
184,629,"Evanston, IL",City,IL,IL,Chicago-Naperville-Elgin,Cook County,750793.0,757648.0,760333.0,760313.0,759305.0,758076.0,755856.0,754216.0,754298.0,754766.0,755798.0,756903.0,754806.0,753732.0,752570.0,751984.0,750429.0,749602.0,750106.0,750109.0,748468.0,746183.0,743861.0,741338.0,739523.0,737185.0,735897.0,735250.0,735621.0,735620.0,2018,16,31,co,Northern - Cook,Completely Urban,0.0,5180493,2455.0,57.0,1266.0,14732.0,2791.0,327.0,1766.0,493.0,47.4,1.1,24.4,284.4,53.9,6.3,34.1,9.5


## Let's Merge all our dataset:


In [285]:
#Let's Merge all our dataset Crime and the City, the dataset income merged with the dataset of population
dataF_all = pd.merge(df_city_crime_merge, df_income_population_merged, how="inner", on="city_name")
dataF_all.head()

,SizeRank,city_name,RegionType,StateName,State,Metro,county_name,1/31/2018,2/28/2018,3/31/2018,4/30/2018,5/31/2018,6/30/2018,7/31/2018,8/31/2018,9/30/2018,10/31/2018,11/30/2018,12/31/2018,1/31/2019,2/28/2019,3/31/2019,4/30/2019,5/31/2019,6/30/2019,7/31/2019,8/31/2019,9/30/2019,10/31/2019,11/30/2019,12/31/2019,1/31/2020,2/29/2020,3/31/2020,4/30/2020,5/31/2020,6/30/2020,year,id,fips_number,judicial_circuit,region,community_type,percent_rural,population_x,aggravated_assault,arson,burglary,larceny_theft,motor_vehicle_theft,murder,robbery,sexual_assault,aggravated_assault_rate,arson_rate,burglary_rate,larceny_theft_rate,motor_vehicle_theft_rate,murder_rate,robbery_rate,sexual_assault_rate,population_y,city_income
0,3,"Chicago, IL",City,IL,IL,Chicago-Naperville-Elgin,Cook County,463386.0,465437.0,464958.0,464273.0,464259.0,465020.0,464935.0,464867.0,465305.0,465988.0,466946.0,467162.0,465606.0,464067.0,463288.0,463675.0,464009.0,463854.0,464049.0,463761.0,463247.0,462900.0,462366.0,461987.0,462121.0,462767.0,463652.0,463953.0,464035.0,464325.0,2018,16,31,co,Northern - Cook,Completely Urban,0.0,5180493,2455.0,57.0,1266.0,14732.0,2791.0,327.0,1766.0,493.0,47.4,1.1,24.4,284.4,53.9,6.3,34.1,9.5,2722389.0,20175.0
1,532,"Cicero, IL",City,IL,IL,Chicago-Naperville-Elgin,Cook County,200197.0,201097.0,201169.0,202682.0,204383.0,205909.0,207135.0,209094.0,211084.0,212932.0,214843.0,215709.0,216468.0,217232.0,217931.0,218303.0,218121.0,218268.0,218645.0,218506.0,218253.0,217890.0,217622.0,217427.0,217595.0,217717.0,218046.0,217975.0,218267.0,218485.0,2018,16,31,co,Northern - Cook,Completely Urban,0.0,5180493,2455.0,57.0,1266.0,14732.0,2791.0,327.0,1766.0,493.0,47.4,1.1,24.4,284.4,53.9,6.3,34.1,9.5,84354.0,12489.0
2,577,"Arlington Heights, IL",City,IL,IL,Chicago-Naperville-Elgin,Cook County,480732.0,482346.0,482235.0,481389.0,481597.0,481500.0,480310.0,478409.0,477775.0,477481.0,478557.0,478652.0,477552.0,475912.0,474600.0,474567.0,473892.0,473217.0,472545.0,472095.0,471196.0,470247.0,468855.0,467828.0,466961.0,466909.0,467159.0,467006.0,466924.0,467078.0,2018,16,31,co,Northern - Cook,Completely Urban,0.0,5180493,2455.0,57.0,1266.0,14732.0,2791.0,327.0,1766.0,493.0,47.4,1.1,24.4,284.4,53.9,6.3,34.1,9.5,76024.0,33544.0
3,629,"Evanston, IL",City,IL,IL,Chicago-Naperville-Elgin,Cook County,750793.0,757648.0,760333.0,760313.0,759305.0,758076.0,755856.0,754216.0,754298.0,754766.0,755798.0,756903.0,754806.0,753732.0,752570.0,751984.0,750429.0,749602.0,750106.0,750109.0,748468.0,746183.0,743861.0,741338.0,739523.0,737185.0,735897.0,735250.0,735621.0,735620.0,2018,16,31,co,Northern - Cook,Completely Urban,0.0,5180493,2455.0,57.0,1266.0,14732.0,2791.0,327.0,1766.0,493.0,47.4,1.1,24.4,284.4,53.9,6.3,34.1,9.5,75658.0,33645.0
4,763,"Palatine, IL",City,IL,IL,Chicago-Naperville-Elgin,Cook County,432637.0,434100.0,433413.0,432465.0,432526.0,433605.0,433496.0,433145.0,432817.0,432815.0,433360.0,433573.0,432662.0,431145.0,430291.0,430123.0,429616.0,428541.0,427867.0,427341.0,426697.0,426064.0,425144.0,424068.0,423159.0,423104.0,423267.0,423129.0,422898.0,422783.0,2018,16,31,co,Northern - Cook,Completely Urban,0.0,5180493,2455.0,57.0,1266.0,14732.0,2791.0,327.0,1766.0,493.0,47.4,1.1,24.4,284.4,53.9,6.3,34.1,9.5,69387.0,30661.0


In [286]:
dataF_all.shape

(459, 63)

In [287]:
dataF_all.isnull().sum()

SizeRank                    0
city_name                   0
RegionType                  0
StateName                   0
State                       0
Metro                       0
county_name                 0
1/31/2018                   0
2/28/2018                   0
3/31/2018                   0
4/30/2018                   0
5/31/2018                   0
6/30/2018                   0
7/31/2018                   0
8/31/2018                   0
9/30/2018                   0
10/31/2018                  0
11/30/2018                  0
12/31/2018                  0
1/31/2019                   0
2/28/2019                   0
3/31/2019                   0
4/30/2019                   0
5/31/2019                   0
6/30/2019                   0
7/31/2019                   0
8/31/2019                   0
9/30/2019                   0
10/31/2019                  0
11/30/2019                  0
12/31/2019                  0
1/31/2020                   0
2/29/2020                   0
3/31/2020 

In [288]:
dataF_all.describe()

,SizeRank,1/31/2018,2/28/2018,3/31/2018,4/30/2018,5/31/2018,6/30/2018,7/31/2018,8/31/2018,9/30/2018,10/31/2018,11/30/2018,12/31/2018,1/31/2019,2/28/2019,3/31/2019,4/30/2019,5/31/2019,6/30/2019,7/31/2019,8/31/2019,9/30/2019,10/31/2019,11/30/2019,12/31/2019,1/31/2020,2/29/2020,3/31/2020,4/30/2020,5/31/2020,6/30/2020,year,id,fips_number,percent_rural,population_x,aggravated_assault,arson,burglary,larceny_theft,motor_vehicle_theft,murder,robbery,sexual_assault,aggravated_assault_rate,arson_rate,burglary_rate,larceny_theft_rate,motor_vehicle_theft_rate,murder_rate,robbery_rate,sexual_assault_rate,population_y,city_income
count,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.0,459.000000,459.000000,459.000000,4.590000e+02,459.000000,459.000000,459.000000,459.00000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,4.590000e+02,459.000000
mean,12824.860566,214153.089325,214642.845316,215063.152505,215468.481481,215927.758170,216346.071895,216612.084967,216807.962963,217124.954248,217419.562092,217775.928105,218053.579521,218297.897603,218583.217865,218763.657952,218953.784314,219224.298475,219483.056645,219829.614379,220021.511983,220213.982571,220367.962963,220413.516340,220476.930283,220532.398693,220613.895425,220771.688453,220887.821351,220875.095861,220852.949891,2018.0,49.766885,98.533769,26.875599,8.170244e+05,440.740741,9.259259,210.041394,2289.24183,367.291939,44.864924,242.124183,70.374728,98.143573,1.549237,39.958170,267.351416,16.871895,2.441394,14.298475,6.583224,1.596695e+04,20517.551198
std,8708.744943,123892.677891,124293.106503,124454.591550,124378.822614,124399.317105,124557.666483,124566.138492,124549.648326,124587.335516,124601.147680,124575.349782,124367.640027,123972.938474,123770.832509,123550.480919,123438.677328,123220.600096,122908.140305,122667.094699,122370.303977,122087.640260,121726.952998,121408.249333,121147.245682,120906.092014,120600.990498,120384.649783,120182.826469,120016.362193,119886.537174,0.0,29.647863,59.295725,24.295484,1.676390e+06,777.749660,18.450107,405.430049,4764.36323,923.060037,107.548970,580.682004,161.111976,73.814172,3.337450,30.075603,160.277573,17.279327,2.816473,13.432112,4.630217,1.283434e+05,6203.618122
min,3.000000,39612.000000,39543.000000,39565.000000,39544.000000,39973.000000,40170.000000,40196.000000,40049.000000,39982.000000,39983.000000,39872.000000,39611.000000,39480.000000,39738.000000,40342.000000,40824.000000,40994.000000,41006.000000,41413.000000,41668.000000,41856.000000,41835.000000,41813.000000,42073.000000,42206.000000,42164.000000,41823.000000,41566.000000,41351.000000,41268.000000,2018.0,1.000000,1.000000,0.000000,5.427000e+03,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.130000e+02,8574.000000
25%,4675.500000,135652.500000,135777.000000,136170.500000,136385.000000,136971.000000,137423.000000,137854.500000,137980.500000,137858.500000,138042.000000,138233.000000,138318.000000,138602.000000,138604.000000,138684.500000,138757.500000,139366.000000,139558.000000,139803.500000,140314.500000,140882.500000,141326.500000,141368.500000,141860.000000,142182.500000,141814.500000,141503.000000,141303.500000,141108.000000,141097.000000,2018.0,19.000000,37.000000,7.900000,3.846300e+04,24.000000,0.000000,12.000000,60.00000,3.000000,0.000000,2.000000,1.000000,47.400000,0.000000,17.700000,159.100000,4.400000,0.000000,3.900000,3.100000,6.880000e+02,16829.500000
50%,11959.000000,183492.000000,183665.000000,183902.000000,184174.000000,184828.000000,185128.000000,185143.000000,186044.000000,185974.000000,186686.

In [289]:
dataF_all.info

<bound method DataFrame.info of      SizeRank               city_name RegionType StateName State  \
0           3             Chicago, IL       City        IL    IL   
1         532              Cicero, IL       City        IL    IL   
2         577   Arlington Heights, IL       City        IL    IL   
3         629            Evanston, IL       City        IL    IL   
4         763            Palatine, IL       City        IL    IL   
5         939              Berwyn, IL       City        IL    IL   
6         947            Glenview, IL       City        IL    IL   
7        1049            Oak Park, IL       City        IL    IL   
8        1414            Wheeling, IL       City        IL    IL   
9        1451          Northbrook, IL       City        IL    IL   
10       1485          Park Ridge, IL       City        IL    IL   
11       1805               Niles, IL       City        IL    IL   
12       1975             Lansing, IL       City        IL    IL   
13       2178   

# See the average houses prices of each city, the equation and their variation the price minimun  and the price maximale in the date columns

In [290]:
# select all the date columns
date_col  = ['1/31/2018','2/28/2018', '3/31/2018' ,'4/30/2018' ,'5/31/2018' ,'6/30/2018' ,'7/31/2018', '8/31/2018','9/30/2018' ,'10/31/2018' ,'11/30/2018' ,'12/31/2018','1/31/2019' ,'2/28/2019' ,'3/31/2019' ,'4/30/2019' ,'5/31/2019' ,'6/30/2019' ,'7/31/2019' ,'8/31/2019' ,'9/30/2019' ,'10/31/2019' ,'11/30/2019' ,'12/31/2019' ,'1/31/2020' ,'2/29/2020','3/31/2020' ,'4/30/2020' ,'5/31/2020' ,'6/30/2020']

In [291]:
# the average of the house price per date
dataF_all['house_price_mean'] = dataF_all[date_col].mean(axis=1)
# the rat of variation of  the house price per date
dataF_all['rate_price_variation'] =  (dataF_all[date_col].std(axis=1)/dataF_all[date_col].mean(axis=1))*100
# the house  price maximale of each city  per date
dataF_all['max_house_price'] = dataF_all[date_col].max(axis = 1)
# the house  price minimale of each city  per date
dataF_all['min_house_price'] = dataF_all[date_col].min(axis = 1)
# the range of  each city  per date
dataF_all['range_house_price'] = dataF_all[date_col].max(axis = 1)- dataF_all[date_col].min(axis = 1)
dataF_all.head()

,SizeRank,city_name,RegionType,StateName,State,Metro,county_name,1/31/2018,2/28/2018,3/31/2018,4/30/2018,5/31/2018,6/30/2018,7/31/2018,8/31/2018,9/30/2018,10/31/2018,11/30/2018,12/31/2018,1/31/2019,2/28/2019,3/31/2019,4/30/2019,5/31/2019,6/30/2019,7/31/2019,8/31/2019,9/30/2019,10/31/2019,11/30/2019,12/31/2019,1/31/2020,2/29/2020,3/31/2020,4/30/2020,5/31/2020,6/30/2020,year,id,fips_number,judicial_circuit,region,community_type,percent_rural,population_x,aggravated_assault,arson,burglary,larceny_theft,motor_vehicle_theft,murder,robbery,sexual_assault,aggravated_assault_rate,arson_rate,burglary_rate,larceny_theft_rate,motor_vehicle_theft_rate,murder_rate,robbery_rate,sexual_assault_rate,population_y,city_income,house_price_mean,rate_price_variation,max_house_price,min_house_price,range_house_price
0,3,"Chicago, IL",City,IL,IL,Chicago-Naperville-Elgin,Cook County,463386.0,465437.0,464958.0,464273.0,464259.0,465020.0,464935.0,464867.0,465305.0,465988.0,466946.0,467162.0,465606.0,464067.0,463288.0,463675.0,464009.0,463854.0,464049.0,463761.0,463247.0,462900.0,462366.0,461987.0,462121.0,462767.0,463652.0,463953.0,464035.0,464325.0,2018,16,31,co,Northern - Cook,Completely Urban,0.0,5180493,2455.0,57.0,1266.0,14732.0,2791.0,327.0,1766.0,493.0,47.4,1.1,24.4,284.4,53.9,6.3,34.1,9.5,2722389.0,20175.0,464206.600000,0.272127,467162.0,461987.0,5175.0
1,532,"Cicero, IL",City,IL,IL,Chicago-Naperville-Elgin,Cook County,200197.0,201097.0,201169.0,202682.0,204383.0,205909.0,207135.0,209094.0,211084.0,212932.0,214843.0,215709.0,216468.0,217232.0,217931.0,218303.0,218121.0,218268.0,218645.0,218506.0,218253.0,217890.0,217622.0,217427.0,217595.0,217717.0,218046.0,217975.0,218267.0,218485.0,2018,16,31,co,Northern - Cook,Completely Urban,0.0,5180493,2455.0,57.0,1266.0,14732.0,2791.0,327.0,1766.0,493.0,47.4,1.1,24.4,284.4,53.9,6.3,34.1,9.5,84354.0,12489.0,213632.833333,2.974510,218645.0,200197.0,18448.0
2,577,"Arlington Heights, IL",City,IL,IL,Chicago-Naperville-Elgin,Cook County,480732.0,482346.0,482235.0,481389.0,481597.0,481500.0,480310.0,478409.0,477775.0,477481.0,478557.0,478652.0,477552.0,475912.0,474600.0,474567.0,473892.0,473217.0,472545.0,472095.0,471196.0,470247.0,468855.0,467828.0,466961.0,466909.0,467159.0,467006.0,466924.0,467078.0,2018,16,31,co,Northern - Cook,Completely Urban,0.0,5180493,2455.0,57.0,1266.0,14732.0,2791.0,327.0,1766.0,493.0,47.4,1.1,24.4,284.4,53.9,6.3,34.1,9.5,76024.0,33544.0,474517.533333,1.157395,482346.0,466909.0,15437.0
3,629,"Evanston, IL",City,IL,IL,Chicago-Naperville-Elgin,Cook County,750793.0,757648.0,760333.0,760313.0,759305.0,758076.0,755856.0,754216.0,754298.0,754766.0,755798.0,756903.0,754806.0,753732.0,752570.0,751984.0,750429.0,749602.0,750106.0,750109.0,748468.0,746183.0,743861.0,741338.0,739523.0,737185.0,735897.0,735250.0,735621.0,735620.0,2018,16,31,co,Northern - Cook,Completely Urban,0.0,5180493,2455.0,57.0,1266.0,14732.0,2791.0,327.0,1766.0,493.0,47.4,1.1,24.4,284.4,53.9,6.3,34.1,9.5,75658.0,33645.0,749686.300000,1.073185,760333.0,735250.0,25083.0
4,763,"Palatine, IL",City,IL,IL,Chicago-Naperville-Elgin,Cook County,432637.0,434100.0,433413.0,432465.0,432526.0,433605.0,433496.0,433145.0,432817.0,432815.0,433360.0,433573.0,432662.0,431145.0,430291.0,430123.0,429616.0,428541.0,427867.0,427341.0,426697.0,426064.0,425144.0,424068.0,423159.0,423104.0,423267.0,423129.0,422898.0,422783.0,2018,16,31,co,Northern - Cook,Completely Urban,0.0,5180493,2455.0,57.0,1266.0,14732.0,2791.0,327.0,1766.0,493.0,47.4,1.1,24.4,284.4,53.9,6.3,34.1,9.5,69387.0,30661.0,429195.033333,0.967883,434100.0,422783.0,11317.0


In [292]:
dataF_all.dtypes

SizeRank                      int64
city_name                    object
RegionType                   object
StateName                    object
State                        object
Metro                        object
county_name                  object
1/31/2018                   float64
2/28/2018                   float64
3/31/2018                   float64
4/30/2018                   float64
5/31/2018                   float64
6/30/2018                   float64
7/31/2018                   float64
8/31/2018                   float64
9/30/2018                   float64
10/31/2018                  float64
11/30/2018                  float64
12/31/2018                  float64
1/31/2019                   float64
2/28/2019                   float64
3/31/2019                   float64
4/30/2019                   float64
5/31/2019                   float64
6/30/2019                   float64
7/31/2019                   float64
8/31/2019                   float64
9/30/2019                   

In [293]:
dataF_all.isnull().sum()

SizeRank                    0
city_name                   0
RegionType                  0
StateName                   0
State                       0
Metro                       0
county_name                 0
1/31/2018                   0
2/28/2018                   0
3/31/2018                   0
4/30/2018                   0
5/31/2018                   0
6/30/2018                   0
7/31/2018                   0
8/31/2018                   0
9/30/2018                   0
10/31/2018                  0
11/30/2018                  0
12/31/2018                  0
1/31/2019                   0
2/28/2019                   0
3/31/2019                   0
4/30/2019                   0
5/31/2019                   0
6/30/2019                   0
7/31/2019                   0
8/31/2019                   0
9/30/2019                   0
10/31/2019                  0
11/30/2019                  0
12/31/2019                  0
1/31/2020                   0
2/29/2020                   0
3/31/2020 

# The geograpical coordinate of each city in Illinois

In [317]:
data_geo = pd.DataFrame({'city_name':[''],'Latitude':[''],'Longitude':['']})
city_names,long,lat = [],[],[]
for city_name in dataF_all['city_name']:
    print(city_name)
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(city_name)
    long.append( location.longitude)
    lat.append(location.latitude)
    city_names.append(city_name)
    print('The geograpical coordinate of each city in Illinois are {}, {}.'.format(latitude, longitude))

Chicago, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Cicero, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Arlington Heights, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Evanston, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Palatine, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Berwyn, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Glenview, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Oak Park, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Wheeling, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Northbrook, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Park Ridge, IL
The geograpical coordinate of each city in Illino

The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Wayne, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Springfield, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Rochester, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Chatham, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Sherman, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Riverton, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Pawnee, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Pleasant Plains, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Auburn, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Williamsville, IL
The geograpical coordinate of each city in Illinois are 40.8644

The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Quincy, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Payson, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Liberty, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Mendon, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Plainville, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Clayton, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Loraine, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Golden, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Sycamore, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Sandwich, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.

The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Highland, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Troy, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Wood River, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Maryville, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Worden, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Alhambra, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Hartford, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Livingston, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Williamson, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Carbondale, IL
The geograpical coordinate of each city in Illinois are 40.864483, 

The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Woodhull, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Cambridge, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Andover, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Alpha, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Canton, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Lewistown, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Farmington, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Cuba, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Astoria, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Avon, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
V

The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Plymouth, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Carthage, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Augusta, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Henry, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Hopewell, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
New Boston, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Viola, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Aledo, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Granville, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.9612528.
Magnolia, IL
The geograpical coordinate of each city in Illinois are 40.864483, -90.961252

In [314]:
data_geo =pd.DataFrame(dict(city_names=city_names,long=long,lat=lat))
data_geo

,city_names,long,lat
0,"Chicago, IL",-87.624421,41.875562
1,"Cicero, IL",-87.753945,41.845588
2,"Arlington Heights, IL",-87.980216,42.081156
3,"Evanston, IL",-87.693046,42.044739
4,"Palatine, IL",-88.033689,42.110578
5,"Berwyn, IL",-87.793668,41.850587
6,"Glenview, IL",-87.811404,42.070066
7,"Oak Park, IL",-87.788762,41.887814
8,"Wheeling, IL",-87.929281,42.125090
9,"Northbrook, IL",-84.581479,39.246087
